In [10]:
import pandas as pd
import geopandas as gpd
import osmnx as osm
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from pyproj import Transformer
from shapely.geometry import *
import networkx as nx

In [7]:
station_data = pd.read_csv('../Data/02_Processed_Data/Station_data_with_community.csv')
station_data = gpd.GeoDataFrame(station_data, geometry=gpd.points_from_xy(station_data['lon'], station_data['lat'], crs='epsg:4326'))
station_data = osm.project_gdf(station_data)
central_london = osm.io.load_graphml('../Data/02_Processed_Data/Central_london.osm')


In [8]:
tube_date = pd.read_csv('../Data/02_Processed_Data/tube_data.csv')

# Tube Station Distance

In [11]:
# Create the walking network graph for London
G = osm.graph_from_place("London, United Kingdom", network_type='walk')


In [12]:
# Convert coordinates to nearest nodes in the graph
station_data['osmid'] = station_data.apply(lambda row: osm.distance.nearest_nodes(G, row['lon'], row['lat']), axis=1)



KeyError: 'lon'

In [14]:
tube_date['osmid'] = tube_date.apply(lambda row: osm.distance.nearest_nodes(G, row['long'], row['lat']), axis=1)

In [17]:
def get_closest_metro(bike_osmid, metro_osmids):
    distances = []
    for metro_osmid in metro_osmids:
        try:
            distance = nx.shortest_path_length(G, source=bike_osmid, target=metro_osmid, weight='length')
            distances.append((metro_osmid, distance))
        except nx.NetworkXNoPath:
            continue
    if distances:
        return min(distances, key=lambda x: x[1])
    return None

# Calculate the closest metro station for each bike station
closest_metro = []
for _, bike_row in station_data.iterrows():
    closest = get_closest_metro(bike_row['osmid'], tube_date['osmid'].tolist())
    if closest:
        closest_metro.append((bike_row['commonName'], closest[0], closest[1]))
    else:
        closest_metro.append((bike_row['commonName'], None, None))

# Convert results to DataFrame
closest_metro_df = pd.DataFrame(closest_metro, columns=['bike_station', 'closest_metro_osmid', 'distance_meters'])

# Merge with metro stations to get the metro station names
closest_metro_df = closest_metro_df.merge(tube_date[['name', 'osmid']], left_on='closest_metro_osmid', right_on='osmid', how='left')

# Final DataFrame with bike stations, closest metro station, and walking distance
closest_metro_df = closest_metro_df[['bike_station', 'name', 'distance_meters']]
closest_metro_df.columns = ['bike_station', 'closest_metro_station', 'distance_meters']

# Display the final result
closest_metro_df


KeyboardInterrupt: 

In [20]:
station_data

,commonName,lat,lon,NbDocks,geometry,Station_number,Community,osmid
0,"River Street , Clerkenwell",51.529163,-0.109970,19,POINT (700457.690 5712632.137),1023,1,25455206
1,"Christopher Street, Liverpool Street",51.521283,-0.084605,32,POINT (702251.641 5711825.859),1012,1,419242224
2,"St. Chad's Street, King's Cross",51.530059,-0.120973,23,POINT (699690.724 5712701.655),1013,0,1511438692
3,"Sedding Street, Sloane Square",51.493130,-0.156876,27,POINT (697360.546 5708498.365),3420,3,8349288276
4,"Broadcasting House, Marylebone",51.518117,-0.144228,18,POINT (698129.901 5711310.670),3424,0,1684410129
...,...,...,...,...,...,...,...,...
365,"Soho Square , Soho",51.515631,-0.132328,57,POINT (698966.191 5711066.572),1052,0,107324
366,"Upper Grosvenor Street, Mayfair",51.510130,-0.155757,18,POINT (697364.716 5710391.488),10621,0,1650513133
367,"Park Lane, Mayfair",51.505426,-0.150817,18,POINT (697727.817 5709881.825),300093,3,1706915439
368,"Pall Mall East, West End",51.507770,-0.130699,22,POINT (699113.482 5710197.001),1153,3,26630839


In [22]:
# Convert coordinates to nearest nodes in the graph for all stations at once
# bike_station_nodes = osm.distance.nearest_nodes(G, station_data['lon'], station_data['lat'])
# metro_station_nodes = osm.distance.nearest_nodes(G, tube_date['long'], tube_date['lat'])
bike_station_nodes = station_data['osmid'].tolist()


In [23]:
metro_station_nodes = tube_date['osmid'].tolist()

In [24]:
import numpy as np

# Precompute the shortest path lengths between all pairs of bike and metro station nodes
path_lengths = np.zeros((len(bike_station_nodes), len(metro_station_nodes)))

for i, bike_node in enumerate(bike_station_nodes):
    for j, metro_node in enumerate(metro_station_nodes):
        try:
            path_lengths[i, j] = nx.shortest_path_length(G, source=bike_node, target=metro_node, weight='length')
        except nx.NetworkXNoPath:
            path_lengths[i, j] = np.inf  # If no path exists, set the distance to infinity

# Find the closest metro station for each bike station
closest_indices = path_lengths.argmin(axis=1)
closest_distances = path_lengths.min(axis=1)

# Create a DataFrame with the results
closest_metro_df = pd.DataFrame({
    'bike_station': station_data['commonName'],
    'closest_metro_station': tube_date['name'].iloc[closest_indices].values,
    'distance_meters': closest_distances
})


KeyboardInterrupt: 

In [34]:

# Convert to GeoDataFrames
station_data['geometry'] = station_data.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
tube_date['geometry'] = tube_date.apply(lambda row: Point(row['long'], row['lat']), axis=1)

station_gdf = gpd.GeoDataFrame(station_data, geometry='geometry')
tube_gdf = gpd.GeoDataFrame(tube_date, geometry='geometry')

# Set the CRS (Coordinate Reference System) to WGS84 (EPSG:4326)
# station_gdf.set_crs(epsg=4326, inplace=True)
station_gdf.projection = 'epsg:4326'
#tube_gdf.set_crs(epsg=4326, inplace=True)
tube_gdf.projection = 'epsg:4326'

# Perform a spatial join with a buffer of 1 km
tube_gdf_buffered = tube_gdf.copy()
tube_gdf_buffered['geometry'] = tube_gdf_buffered.geometry.buffer(0.01)  # Buffer by 0.01 degrees ~ 1km

# Spatial join (only keep those within 1km)
potential_matches = gpd.sjoin(station_gdf, tube_gdf_buffered, how='inner', op='intersects')
print(potential_matches)
# Create the walking network graph for London
#G = osm.graph_from_place("London, United Kingdom", network_type='walk')

# Ensure that coordinates are being correctly mapped to OSM nodes
potential_matches['bike_osmid'] = osm.distance.nearest_nodes(G, potential_matches['lon'], potential_matches['lat_left'])
potential_matches['metro_osmid'] = osm.distance.nearest_nodes(G, potential_matches['lon'], potential_matches['lat_left'])

# Efficient function to calculate the closest metro station
def get_closest_metro(bike_osmid, metro_osmids):
    distances = []
    for metro_osmid in metro_osmids:
        try:
            distance = nx.shortest_path_length(G, source=bike_osmid, target=metro_osmid, weight='length')
            distances.append((metro_osmid, distance))
        except nx.NetworkXNoPath:
            continue
    if distances:
        return min(distances, key=lambda x: x[1])
    return None

# Calculate the closest metro station for each bike station
closest_metro = []
grouped = potential_matches.groupby('commonName')

for bike_station, group in grouped:
    bike_osmid = group['bike_osmid'].iloc[0]
    metro_osmids = group['metro_osmid'].tolist()
    closest = get_closest_metro(bike_osmid, metro_osmids)
    if closest:
        closest_metro.append((bike_station, closest[0], closest[1]))
    else:
        closest_metro.append((bike_station, None, None))

# Convert results to DataFrame
closest_metro_df = pd.DataFrame(closest_metro, columns=['bike_station', 'closest_metro_osmid', 'distance_meters'])

# for the 

# Merge with metro stations to get the metro station names
closest_metro_df = closest_metro_df.merge(tube_gdf[['Station', 'osmid']], left_on='closest_metro_osmid', right_on='osmid', how='left')

# Final DataFrame with bike stations, closest metro station, and walking distance
closest_metro_df = closest_metro_df[['bike_station', 'Station', 'distance_meters']]
closest_metro_df.columns = ['bike_station', 'closest_metro_station', 'distance_meters']


/Users/juliuskrug/anaconda3/envs/ox/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/0m/swmphxyn7jncj7fk8v1qrj5c0000gn/T/ipykernel_61265/2246068298.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:32630
Right CRS: None

  potential_matches = gpd.sjoin(station_gdf, tube_gdf_buffered, how='inner', op='intersects')


                               commonName   lat_left       lon  NbDocks  \
0              River Street , Clerkenwell  51.529163 -0.109970       19   
1    Christopher Street, Liverpool Street  51.521283 -0.084605       32   
1    Christopher Street, Liverpool Street  51.521283 -0.084605       32   
1    Christopher Street, Liverpool Street  51.521283 -0.084605       32   
1    Christopher Street, Liverpool Street  51.521283 -0.084605       32   
..                                    ...        ...       ...      ...   
368              Pall Mall East, West End  51.507770 -0.130699       22   
368              Pall Mall East, West End  51.507770 -0.130699       22   
369           Green Park Station, Mayfair  51.506613 -0.142844       28   
369           Green Park Station, Mayfair  51.506613 -0.142844       28   
369           Green Park Station, Mayfair  51.506613 -0.142844       28   

                  geometry  Station_number  Community  osmid_left  \
0    POINT (-0.110 51.529)    

In [35]:
closest_metro_df

,bike_station,closest_metro_station,distance_meters
0,"Abbey Orchard Street, Westminster",NaN,0
1,"Abingdon Green, Westminster",NaN,0
2,"Albany Street, The Regent's Park",NaN,0
3,"Albert Embankment, Vauxhall",NaN,0
4,"Albert Gate, Hyde Park",NaN,0
...,...,...,...
349,"Wood Street, Guildhall",NaN,0
350,"Woodstock Street, Mayfair",NaN,0
351,"Wormwood Street, Liverpool Street",Liverpool Street,0
352,"Worship Street, Shoreditch",NaN,0


In [16]:
station_data

,commonName,lat,lon,NbDocks,geometry,Station_number,Community,osmid
0,"River Street , Clerkenwell",51.529163,-0.109970,19,POINT (700457.690 5712632.137),1023,1,25455206
1,"Christopher Street, Liverpool Street",51.521283,-0.084605,32,POINT (702251.641 5711825.859),1012,1,419242224
2,"St. Chad's Street, King's Cross",51.530059,-0.120973,23,POINT (699690.724 5712701.655),1013,0,1511438692
3,"Sedding Street, Sloane Square",51.493130,-0.156876,27,POINT (697360.546 5708498.365),3420,3,8349288276
4,"Broadcasting House, Marylebone",51.518117,-0.144228,18,POINT (698129.901 5711310.670),3424,0,1684410129
...,...,...,...,...,...,...,...,...
365,"Soho Square , Soho",51.515631,-0.132328,57,POINT (698966.191 5711066.572),1052,0,107324
366,"Upper Grosvenor Street, Mayfair",51.510130,-0.155757,18,POINT (697364.716 5710391.488),10621,0,1650513133
367,"Park Lane, Mayfair",51.505426,-0.150817,18,POINT (697727.817 5709881.825),300093,3,1706915439
368,"Pall Mall East, West End",51.507770,-0.130699,22,POINT (699113.482 5710197.001),1153,3,26630839
